V:2022/04/04

Note: Pseudo code is shown in **bold** in the cell before each piece of code.

# Elimination of the anthropogenic noise: `antropo`

The code shown is this notebook is aimed at the removal of the anthropogenic signals detected in the measurements recorded at the station. The anthropogenic noise appears as narrow peaks in the signal spectrum. These signals appear mainly in three bands: {14.44,15.00}, {15.00,15.35}, {16.50,16.90} Hz.
Due to the proximity of the first two bands, they have been merged into one only band, so the bands {14.55,15.35} Hz and {16.50,16.90} Hz have been considered in this program.

A visual inspection of the 10-min spectra (filtered and unfiltered) can be done with this Jupyter notebook.

## Packages and inputs

**Packages used in this program. `ipywidgets` is not used in the script**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
%matplotlib inline

**Input and output paths**

In [2]:
# INPUTS
year="2015"
month="1503"
pathg='S_N_FD/'           # Have to be specified        

We define the year and month that we are studying. Next we define the path where the amplitude spectra files are located.

**Station parameters. The frecuency increment is calculated. The calibrated frequency list is defined**

In [3]:
fm=256      # Sampling frequency
bandcal=np.array([6,25],dtype=float)   # Calibrated frequency band
nwindow=2**13
nhour=fm*3600
df=float(fm)/float(nwindow)
fre=np.arange(bandcal[0],bandcal[1]+df,df)   # calibrated frequec
nf=len(fre)

**Anthropogenic noise and reference frequency bands**

In [4]:
listafrecantro=np.array([[[14.55,15.35],[13.5,13.9]],\
                         [[16.50,16.90],[15.8,16.4]]])

At this point, the frequency bands with anthropogenic noise are defined. For each one, we must also define a reference band that is free of anthropogenic noise and with similar features to that containing the noise.

These bands are valid for all the months, although in specific cases the starting and ending points may vary. There are two types of antropoghenic noise: one that has been associated with the electric generator of the mountain hut near the station and another one that seems to be associated with the Central Europe railway network. The first noise sometimes presents two peaks with very close frequencies, so it is necessary to define a band that encompasses the two subbands.

**The positions of the limits of the anthropogenic noisy and reference frequency bands are located in the calibrated frequency list using the function `pofre`**

In [5]:
def pofre(f,df,fi):
    return int(round((f-fi)/df))

The function `pofre(f,df,fi)` determines the position of a frequency `f` in a list of frequencies with a starting frequency, `fi`, and a frequency increment `df`.

In [6]:
positions=np.array([pofre(f,df,bandcal[0])\
                   for f in listafrecantro.ravel()],dtype=int)
antropos=(positions.reshape(listafrecantro.shape))[:,0]
refepos=(positions.reshape(listafrecantro.shape))[:,1]

Using the `pofre` function, the positions of the frequency band limits (for the noise and reference bands) are located in the calibration frequency list.

**Reading spectra and the saturation percentage for each 10-min interval**

In [7]:
pathmonth=pathg+year+"/"+month+"/"+"SR"+month[0:4]+"_"

fichmedia0=pathmonth+"media"+"_0"
fichmedia1=pathmonth+"media"+"_1"

fichsatper0=pathmonth+"satper"+"_0"
fichsatper1=pathmonth+"satper"+"_1"                               
                               
medidas0=np.genfromtxt(fichmedia0).reshape(-1,nf)
medidas1=np.genfromtxt(fichmedia1).reshape(-1,nf)
satper0=np.genfromtxt(fichsatper0)
satper1=np.genfromtxt(fichsatper1)
nintervm=len(medidas0)

We read the 10-min amplitude spectra.
The saturation percentage file for each 10-min interval is also read, in order to avoid the division by zero in the case of saturated intervals. This is avoided by checking that the interval is not saturated, that is, the saturation percentage is strictly less than 1. The variables involved in saturation carry the infix `satper`.

Since the data are recorded without splitting, they are partitioned using the `nf` variable that corresponds to the total number of frequencies stored in each 10-min interval.

**This function is used to plot the 10-min amplitude spectrum with the help of the package `interact`**

In [8]:
def spectrumNoisy(indi):
    plt.grid()
    plt.xlabel('Frequency (Hz)')
    plt.ylabel(r'Amplitude pT/$\sqrt{Hz}$')
    plt.title('Amplitude spectrum')
    plt.plot(fre,medidas0[indi,:],c='b',label='N-S sensor')
    plt.plot(fre,medidas1[indi,:],c='r',label='E-W sensor')
    plt.legend(loc=1)
    plt.show()

In [9]:
interact(spectrumNoisy,indi=(0,nintervm-1))

interactive(children=(IntSlider(value=71, description='indi', max=143), Output()), _dom_classes=('widget-inter…

<function __main__.spectrumNoisy(indi)>

With the function `interact` from ipywidgets package and the `spectrumNoisy` function, we can display the spectrum, for both sensors and for each 10-min interval of the month that we are analyzing.

**The positions of the limits of the anthropogenic noisy and reference frequency bands are located in the calibrated frequency list using the function `pofre`**

In [10]:
positions=np.array([pofre(f,df,bandcal[0])\
                   for f in listafrecantro.ravel()],dtype=int)
antropos=(positions.reshape(listafrecantro.shape))[:,0]
refepos=(positions.reshape(listafrecantro.shape))[:,1]

Using the `pofre` function, the positions in the calibration band corresponding to the boundary frequencies that appear for both the noise and reference bands are located.

## Filtering functions

**The function `recta` fits a stright line to the values defined by the  array `val` corresponding to the frequencies in the array `freq`.**

In [11]:
def recta(val,frec):
    a=(val[-1]-val[0])/(frec[-1]-frec[0])
    b=val[0]
    valmod=a*(frec-frec[0])+b
    de=np.max((val-valmod)/valmod)+np.min((val-valmod)/valmod)
    return valmod, de

Given a set of amplitude spectrum values, `val`, and its frequency set `frec`, the function `recta` calculates:
- The parameters (slope and intercept) of the line defined by the first and last values in `val` and `frec` and with these parameters calculates the value of the stright line corresponding to each frequency in `frec`. 
- The relative difference (with respect to the value of the line) between the points of the line and the values for each frequency; the maximum and minimum values of these relative differences are found and added up to give a value that we name the filter trigger value.


**The function `antro` corrects the anthropogenic noise in each 10-min  spectrum. The spectra of each interval of 10-min are in the array `medidasantro`. The frequencies of each spectrum are in the array `fre`. The reference and anthropogenic bands have their limits defined in `refpos` and `antropos`. The value `facfil` defines the filter application threshold.**

In [12]:
def antro(medidasantro,fre,refpos,antropos,satper,facfil):
    medidas=np.copy(medidasantro)
    for i in range(len(medidas)):
        if satper[i]<0.9999:
            for j in range(len(antropos)):
                desvia = recta(medidas[i,refpos[j,0]:refpos[j,1]+1],\
                               fre[refpos[j,0]:refpos[j,1]+1])[1]
                resul = recta(medidas[i,antropos[j,0]:antropos[j,1]+1],\
                              fre[antropos[j,0]:antropos[j,1]+1])
                if(resul[1]>facfil*np.abs(desvia)):
                    medidas[i,antropos[j,0]:antropos[j,1]+1]=resul[0]
        else:
            print("Saturated interval ",i)  
    return medidas

**The anthropogenic noise filtering is applied to the whole month and to both sensors**

In [13]:
medcor0 = antro(medidas0,fre,refepos,antropos,satper0,2.5)
medcor1 = antro(medidas1,fre,refepos,antropos,satper1,2.5)

The `antro` function performs the noise correction for the entire month. It starts with a loop that goes through all the 10-minute intervals in the month, `{i, 1, Length [medidas]}`. This is followed by another loop for the entire list of bands with possible anthropogenic noise and reference bands, `{j, 1, Length [anthropos]}`. For each interval and each band the following sentences are executed:
* For the reference band, the reference band trigger value is determined with the function `recta`, and it is stored in `desvia`. Therefore, a single value is used from the output of the `recta` function.
* For the anthropogenic band, the function `recta` gives the following output: the line values and the noise band trigger value.
* The filter operates on the condition that the noise band trigger value is greater than the reference band trigger value multiplied by a factor (this factor is defined by the last argument of the `antro` function). If the filter is trigged, the recorded measurements are modified with the straight line. 
* Finally, the filtered values of the measurements are saved to a file.


## Filtering review

**This function is used to draw the filtered spectrum of each 10-min interval with the help of the `interact` package.**

In [14]:
def spectrumDeNoisy(indi):
    plt.grid()
    plt.xlabel('Frequency (Hz)')
    plt.ylabel(r'Amplitude pT/$\sqrt{Hz}$')
    plt.title('Amplitude spectrum')
    plt.plot(fre,medcor0[indi,:],c='b',label='N-S sensor')
    plt.plot(fre,medcor1[indi,:],c='r',label='E-W sensor')
    plt.legend(loc=1)
    plt.show()

In [15]:
interact(spectrumDeNoisy,indi=(0,nintervm-1))

interactive(children=(IntSlider(value=71, description='indi', max=143), Output()), _dom_classes=('widget-inter…

<function __main__.spectrumDeNoisy(indi)>

## Output

The names of the output files are defined with the new infix `mediaNA`.

**The filtered amplitude spectra is written in the output files.**

In [16]:
fichsal0=open(pathmonth+"mediaNA"+"_0",mode='w')
fichsal1=open(pathmonth+"mediaNA"+"_1",mode='w')

for i in range(nintervm):
    print(*medcor0[i],sep='\n',file=fichsal0)
    print(*medcor1[i],sep='\n',file=fichsal1)

fichsal0.close()
fichsal1.close()